In [9]:
from lxml import etree

In [10]:
from typing import List, Tuple

In [11]:
def load_sentirueval_2016(file_name: str) -> Tuple[List[str], List[str]]:
    texts = []
    labels = []
    with open(file_name, mode='rb') as fp:
        xml_data = fp.read()
    root = etree.fromstring(xml_data)
    for database in root.getchildren():
        if database.tag == 'database':
            for table in database.getchildren():
                if table.tag != 'table':
                    continue
                new_text = None
                new_label = None
                for column in table.getchildren():
                    if column.get('name') == 'text':
                        new_text = str(column.text).strip()
                        if new_label is not None:
                            break
                    elif column.get('name') not in {'id', 'twitid', 'date'}:
                        if new_label is None:
                            label_candidate = str(column.text).strip()
                            if label_candidate in {'0', '1', '-1'}:
                                new_label = 'negative' if label_candidate == '-1' else \
                                    ('positive' if label_candidate == '1' else 'neutral')
                                if new_text is not None:
                                    break
                if (new_text is None) or (new_label is None):
                    raise ValueError('File `{0}` contains some error!'.format(file_name))
                texts.append(new_text)
                labels.append(new_label)
            break
    return texts, labels

In [12]:
texts, labels = load_sentirueval_2016('bank_train_2016.xml')

In [13]:
print('Number of texts is {0}, number of labels is {1}.'.format(len(texts), len(labels)))

Number of texts is 9392, number of labels is 9392.


In [1]:
#1)токенизацию с учётом возможных смайлов (базовая токенизация некорректно
#работает со знаками препинания и прочими неалфавитными и нецифровыми
#символами, из которых как раз и могут состоять тонально значимые смайлы)

In [ ]:
# 2) лемматизацию с учётом контекста, чтобы успешно разрешать морфоомонимию вида
# “​ мы ​ стали лучше программировать​ ” - “​ мы выплавляем больше ​ стали​ ” (для такой
# лемматиции можно использовать, например, библиотеку ​ UDPipe или её адаптацию
# под ​ SpaCy​ )

In [ ]:
# 3) удаление стоп-слов по словарям и/или правилам (например, описанным в виде
# регулярных выражений).

In [15]:
from nltk.tokenize import TweetTokenizer
from rnnmorph.predictor import RNNMorphPredictor

predictor = RNNMorphPredictor(language='ru')
tokenizer = TweetTokenizer()

def drop_junk(s):
    return ' '.join([word for word in tokenizer.tokenize(s) if word.isalpha() or '!' in word or '?' in word \
                     or '(' in word or ')' in word])

def lemmatization(s):
    toks = tokenizer.tokenize(s)
    forms = predictor.predict(toks)
    return ' '.join([f.normal_form for f in forms])

texts, labels = load_sentirueval_2016('bank_train_2016.xml')

texts = list(map(drop_junk, texts))
texts = list(map(lemmatization, texts))

In [16]:
from stop_words import get_stop_words

stop_words = get_stop_words('ru')

In [19]:
import random

In [20]:
for idx in random.choices(list(range(len(texts))), k=20):
    print('{0} => {1}'.format(labels[idx], texts[idx]))

negative => инкассаторский броневик сбербанк попасть в дтп на выезд со двор на ул домодедовский полиция ехать охранять броневик с деньга
neutral => втб рефинансирование кредит другой банк калькулятор
positive => поволжский сбербанк проводить акция для малый бизнес поволжский банк оао сбербанк россия о
positive => россеть быть развивать сотрудничество с россельхозбанк и ростехома август состояться заседание правление оао
neutral => кредитка голд альфа банк как оформить без отказ
negative => я новость экономика сша ввести санкция против банк москва втб и россельхозбанк
neutral => взять кредит в газпромбанк
neutral => акция сбербанк динамик акция сбербанк динамик
neutral => купить квартира в спб ипотека
neutral => фермер взять кредит в россельхозбанк
negative => rt втб прекратить приём консульский сбор за оформление виза сша
neutral => корпоратив втб в формат концерт группа несчастный случай ! руб
positive => сбербанк продлевать срок акция по ипотечный кредит сниженный процентный ставка б

In [23]:
positive_tweets = [texts[idx] for idx in range(len(texts)) if labels[idx] == 'positive']
negative_tweets = [texts[idx] for idx in range(len(texts)) if labels[idx] == 'negative']
neutral_tweets = [texts[idx] for idx in range(len(texts)) if labels[idx] == 'neutral']

In [24]:
for cur in positive_tweets[:5]: print(cur)

главный чтоб банк сбер и втб ! ! !
самый выгодный автокредит в втб
легко можно получить денежный кредит ы втб банка
снижение процентный ставка по кредит на недвижимость сбербанк
в наш сбербанк прийти американец и попросить сфоткаться с он у банкомат я ващий быть в шок и на фотка полюбома получиться смешной xd


In [25]:
for cur in negative_tweets[:5]: print(cur)

обязательно про сбербанк написать ! временами хлёсткий интернет магазин финт выкручивать )
втб и сбер точно операция или запрет на кредит ?
канада ввести санкция против газпромбанк вэба новатэк и российский оборонный компания
rt если сбер и втб попасть под санкция быть жопа
rt ввести санкция против оборонный и сырьевой компания и банк среди он в частность


In [26]:
for cur in neutral_tweets[:5]: print(cur)

взять кредит тюмень альфа банк
мнение о кредитный карта втб
райффайзенбанк снижение ключевой ставка цб на заседание в этот пятница очень маловероятный
современный состояние кредитный поведение в россия сбербанк
оформить краткосрочный кредит оао банк москва


In [29]:
from nltk import word_tokenize

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

In [31]:
vectorizer = CountVectorizer(lowercase=True, tokenizer=word_tokenize)

In [32]:
vectorizer.fit(texts)

C:\Users\PC\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:484: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


CountVectorizer(tokenizer=<function word_tokenize at 0x0000020275ACA1F0>)

In [33]:
print(vectorizer.get_feature_names()[0:20])

['!', '(', ')', '-', '8', ':', ';', '=', '?', 'a', 'abncapital', 'admitad', 'ag', 'agro', 'airlines', 'ajhvf', 'alfa', 'alfabank', 'alfaclick', 'aliexpress']


In [34]:
print(len(vectorizer.get_feature_names()))

6660


In [35]:
X = vectorizer.transform(texts)

In [55]:
print(type(X))

<class 'scipy.sparse.csr.csr_matrix'>


In [37]:
print(texts[0])

взять кредит тюмень альфа банк


In [38]:
print(X[0])

  (0, 419)	1
  (0, 555)	1
  (0, 852)	1
  (0, 2494)	1
  (0, 5979)	1


In [39]:
print(vectorizer.get_feature_names()[6321])

хакамада


In [42]:
print(vectorizer.get_feature_names()[5979])

тюмень


In [44]:
from sklearn.feature_extraction.text import TfidfTransformer

In [46]:
transformer = TfidfTransformer().fit_transform(X)

In [48]:
#X_transformed = transformer.transform(X)

In [49]:
print(transformer[0])

  (0, 5979)	0.7391351947070658
  (0, 2494)	0.22539383399193713
  (0, 852)	0.4219051594577849
  (0, 555)	0.26376023593941833
  (0, 419)	0.3940854703028856


In [51]:
transformer1 = TfidfTransformer().fit(X)

In [56]:
X_transformed = transformer1.transform(X)

In [57]:
print(X_transformed[0])

  (0, 5979)	0.7391351947070658
  (0, 2494)	0.22539383399193713
  (0, 852)	0.4219051594577849
  (0, 555)	0.26376023593941833
  (0, 419)	0.3940854703028856


In [59]:
print(vectorizer.get_feature_names()[6321])

хакамада


In [60]:
print(vectorizer.get_feature_names()[5979])

тюмень


In [62]:
tokens_with_IDF = list(zip(vectorizer.get_feature_names(), transformer1.idf_))

In [63]:
for feature, idf in tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

4.417620 => !
4.805386 => (
4.443938 => )
8.355961 => -
9.049108 => 8
6.020586 => :
8.068278 => ;
8.201810 => =
4.440610 => ?
9.049108 => a
9.454573 => abncapital
9.454573 => admitad
9.049108 => ag
9.049108 => agro
9.454573 => airlines
9.454573 => ajhvf
9.049108 => alfa
8.355961 => alfabank
9.049108 => alfaclick
9.454573 => aliexpress


In [64]:
sorted_tokens_with_IDF = sorted(tokens_with_IDF, key=lambda it: (-it[1], it[0]))

In [65]:
for feature, idf in sorted_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.454573 => abncapital
9.454573 => admitad
9.454573 => airlines
9.454573 => ajhvf
9.454573 => aliexpress
9.454573 => api
9.454573 => app
9.454573 => apps
9.454573 => appstore
9.454573 => autopaymts
9.454573 => back
9.454573 => bankvtb
9.454573 => best
9.454573 => biznews
9.454573 => blackberry
9.454573 => blatt
9.454573 => bump
9.454573 => camp
9.454573 => cash
9.454573 => cetelem


In [66]:
from sklearn.feature_selection import SelectPercentile, chi2

In [67]:
selector = SelectPercentile(chi2, percentile=20)

In [68]:
selector.fit(X_transformed, labels)

SelectPercentile(percentile=20,
                 score_func=<function chi2 at 0x000002020CC344C0>)

In [69]:
selected_tokens_with_IDF = [tokens_with_IDF[idx] for idx in selector.get_support(indices=True)]

In [70]:
print(len(selected_tokens_with_IDF))

1332


In [71]:
for feature, idf in selected_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

4.417620 => !
4.805386 => (
4.443938 => )
6.020586 => :
8.201810 => =
4.440610 => ?
9.049108 => apple
9.454573 => appstore
8.761426 => awards
8.538282 => boerse
9.049108 => brkng
9.049108 => cemitsvetic
9.454573 => come
9.454573 => euro
7.056678 => finance
7.439670 => fitch
8.761426 => ft
7.845135 => ftse
7.151988 => global
9.454573 => hsbc


In [72]:
selected_and_sorted_tokens_with_IDF = sorted(selected_tokens_with_IDF, key=lambda it: (-it[1], it[0]))

In [73]:
for feature, idf in selected_and_sorted_tokens_with_IDF[0:20]: print('{0:.6f} => {1}'.format(idf, feature))

9.454573 => appstore
9.454573 => come
9.454573 => euro
9.454573 => hsbc
9.454573 => london
9.454573 => moneynews
9.454573 => n
9.454573 => nalexandrowa
9.454573 => play
9.454573 => plc
9.454573 => rdx
9.454573 => softkey
9.454573 => spo
9.454573 => автолюбитель
9.454573 => автоматический
9.454573 => автопроверка
9.454573 => адекватный
9.454573 => азс
9.454573 => ай
9.454573 => аккредитовый
